In [1]:
import numpy as np
from astropy.table import Table
import mr_forecast as mr
import pandas as pd
import rebound
import spock
import matplotlib.pyplot as plt
% matplotlib inline
np.random.seed(2)

/Users/alysa/miniconda3/envs/astroconda/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/Users/alysa/miniconda3/envs/astroconda/lib/python3.5/site-packages/xgboost/__init__.py:29: FutureWarning: Python 3.5 support is deprecated; XGBoost will require Python 3.6+ in the near future. Consider upgrading to Python 3.6+.
  FutureWarning)


In [2]:
cumulative_df = pd.read_csv("cumulative_2023.05.12_10.14.28.csv", comment="#").sort_values(by="kepoi_name")
cumulative_df['kepler_name'] = cumulative_df['kepler_name'].str.replace(" ", "")
cumulative_df['kepoi_name'] = cumulative_df['kepoi_name'].str.lstrip('K0').astype(float)

In [3]:
koi_gaia_df = pd.read_csv("cumulative_koi_gaia_dr2.csv", comment="#").sort_values(by="kepoi_name")

In [4]:
koi_gaia_df = pd.merge(koi_gaia_df, cumulative_df, on="kepoi_name", suffixes=('', '_DROP')).filter(regex='^(?!.*_DROP)')

In [5]:
bonomo_df = Table.read("bonomo-2023-table-10.tex", format='ascii.latex').to_pandas()

bonomo_df['M_p'] = bonomo_df['M_p'].str.strip("$")
bonomo_df['e'] = bonomo_df['e'].str.strip("$")

bonomo_df['M_p'].loc[bonomo_df['M_p'].str.contains("pm")] = \
    bonomo_df['M_p'].loc[bonomo_df['M_p'].str.contains("pm")].str.split("pm", expand=True).loc[:,0].str.rstrip("\\")
bonomo_df['M_p'].loc[bonomo_df['M_p'].str.contains("_")] = \
    bonomo_df['M_p'].loc[bonomo_df['M_p'].str.contains("_")].str.split("_", expand=True).loc[:,0]
bonomo_df['M_p'] = bonomo_df['M_p'].str.lstrip("< ")

bonomo_df['e'].loc[bonomo_df['e'].str.contains("pm")] = \
    bonomo_df['e'].loc[bonomo_df['e'].str.contains("pm")].str.split("pm", expand=True).loc[:,0].str.rstrip("\\")
bonomo_df['e'] = bonomo_df['e'].str.lstrip("< ")
bonomo_df['e'].loc[bonomo_df['e'].str.contains("\(")] = \
    bonomo_df['e'].loc[bonomo_df['e'].str.contains("\(")].str.split("\(", expand=True).loc[:,0]

bonomo_df['M_p'] = bonomo_df['M_p'].astype(float)
bonomo_df['M_p'] = bonomo_df['M_p'].astype(float)

In [6]:
merged_df = pd.merge(koi_gaia_df, bonomo_df, left_on="kepler_name", right_on="Name")

drop_cols = ['Name', 'T_c', 'P', 'R_p', 'i', 'Ref', 'K', 'rhop', 'logg_p', 'a', 'Teq', 'F_p']

merged_df = merged_df.drop(columns=drop_cols, axis=1)

In [7]:
merged_df.rename(columns={"M_p":"koi_pmass"})

,kepid,kepoi_name,koi_pdisposition,koi_period,koi_period_err1,koi_period_err2,koi_prad,koi_prad_err1,koi_prad_err2,koi_srad,koi_srad_err1,koi_srad_err2,koi_smass,koi_smass_err1,koi_smass_err2,kepler_name,koi_count,e,koi_pmass
0,6850504,70.01,CANDIDATE,10.854096,4.977000e-06,-4.977000e-06,2.87,0.15,0.07,0.888,0.016,-0.015,0.922,0.043,-0.054,Kepler-20c,5,0.076,11.10
1,6850504,70.02,CANDIDATE,3.696121,2.394000e-06,-2.394000e-06,1.71,0.05,0.03,0.888,0.016,-0.015,0.922,0.043,-0.054,Kepler-20b,5,0.083,9.70
2,6850504,70.03,CANDIDATE,77.611443,1.536000e-04,-1.536000e-04,2.49,0.07,0.05,0.888,0.016,-0.015,0.922,0.043,-0.054,Kepler-20d,5,0.082,13.40
3,6850504,70.04,CANDIDATE,6.098556,2.536000e-05,-2.536000e-05,0.81,0.04,0.03,0.888,0.016,-0.015,0.922,0.043,-0.054,Kepler-20e,5,0.092,0.76
4,6850504,70.05,CANDIDATE,19.577607,1.427000e-04,-1.427000e-04,0.86,0.05,0.03,0.888,0.016,-0.015,0.922,0.043,-0.054,Kepler-20f,5,0.094,1.40
5,11904151,72.01,CANDIDATE,0.837491,3.030000e-07,-3.030000e-07,1.46,0.04,0.03,1.066,0.022,-0.022,0.930,0.071,-0.063,Kepler-10b,2,0,3.26
6,11904151,72.02,CANDIDATE,45.294223,5.600000e-05,-5.600000e-05,2.30,0.06,0.05,1.066,0.022,-0.022,0.930,0.071,-0.063,Kepler-10c,2,0.130,11.40
7,10187017,82.01,CANDIDATE,16.145672,7.440000e-06,-7.440000e-06,2.48,0.04,0.21,0.741,0.011,-0.011,0.766,0.032,-0.038,Kepler-102e,6,0.089,4.70
8,10187017,82.02,CANDIDATE,10.311748,1.226000e-05,-1.226000e-05,1.34,0.02,0.12,0.741,0.011,-0.011,0.766,0.032,-0.038,Kepler-102d,6,0.092,3.00
9,10187017,82.03,CANDIDATE,27.453615,1.077000e-04,-1.077000e-04,0.95,0.11,0.10,0.741,0.011,-0.011,0.766,0.032,-0.038,Kepler-102f,6,0.10,4.30


In [8]:
merged_df.to_csv('cumulative_koi_gaia_bonomo.csv', header=True, index=False)